<a href="https://colab.research.google.com/github/tzf101/BDA-Bangla-Text-Data-Augmentation/blob/main/utils_notebook/BDA_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

### EDA

In [1]:
!pip install bnlp

  Preparing metadata (setup.py) ... done
  Created wheel for bnlp: filename=bnlp-0.8-py3-none-any.whl size=32803 sha256=d49a68a5b1f787d26b4a6e5b9b29cddd7b983d9689de8eb0f3749db73d55ab0f
  Stored in directory: /root/.cache/pip/wheels/64/c7/7e/272284eb8c800fbb5306bdf7904ae9f99a64812fc615f471b2
Successfully built bnlp


In [2]:
!pip install bnlp-toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=d041956b67381fbfea968dfc3fc9b843097428668bff46f1c2013332a76b70f8
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


### Transformer

In [3]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-b6bmiozu
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-b6bmiozu
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=4c3732a161c873b7c4de04b67a0842a82b2fae249aae90bdeb5904eabc1c94e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qovdqz5_/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=575fbef587134863894226ec2dc84bb7b099c2b

In [4]:
!pip install sentencepiece

# Imports

### EDA

In [5]:
import random
import re
from bnlp import BengaliCorpus as corpus
from bnlp import BengaliWord2Vec

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Transformers

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize
import sentencepiece as spm
import torch
from transformers import BertTokenizer  # From Hugging Face's transformers library
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Text Cleaning

In [7]:
class CleanText:
    def __init__(self):
        self.punctuations = set(corpus.punctuations + "‘’")
        self.stopwords = set(corpus.stopwords)

    def remove_digits(self, text):
        return re.sub(r'[০-৯]+\d+', '', text).strip()

    def remove_punctuations(self, text, replace_with=" "):
        for punct in self.punctuations:
            text = text.replace(punct, replace_with)
        return ' '.join(text.split())

    def remove_stopwords(self, text):
        words = text.split()
        return ' '.join(word for word in words if word.lower() not in self.stopwords)

# Get Synonyms

In [8]:
def get_synonyms(word):
    synonyms = set()
    bwv = BengaliWord2Vec()
    similar_words = bwv.get_most_similar_words(word, topn=10)
    for word in similar_words:
        synonyms.add(word[0])
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

# Base Class

In [9]:
class BaseTextAugmentor:
    def __init__(self, clean_text):
        self.clean_text = clean_text

    def augment(self, text):
        raise NotImplementedError("This method should be implemented by subclasses")

# Paraphrase

In [10]:
class BanglaParaphraseAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, model_name="csebuetnlp/banglat5_banglaparaphrase"):
        super().__init__(clean_text)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    def augment(self, text, debug=False):
        input_tokens = self.tokenizer(normalize(text), return_tensors="pt").input_ids
        generated_tokens = self.model.generate(input_tokens)
        decoded_tokens = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        if debug:
            decoded_tokens += '(pp)'
        return decoded_tokens

# BT

In [11]:
class BTAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, modelb2e_name="csebuetnlp/banglat5_nmt_bn_en", modele2b_name="csebuetnlp/banglat5_nmt_en_bn"):
        super().__init__(clean_text)
        self.modelb2e = AutoModelForSeq2SeqLM.from_pretrained(modelb2e_name)
        self.tokenizerb2e = AutoTokenizer.from_pretrained(modelb2e_name, use_fast=False)

        self.modele2b = AutoModelForSeq2SeqLM.from_pretrained(modele2b_name)
        self.tokenizere2b = AutoTokenizer.from_pretrained(modele2b_name, use_fast=False)

    def augment(self, text, debug=False):
        input_ids = self.tokenizerb2e(normalize(text), return_tensors="pt").input_ids
        generated_tokens = self.modelb2e.generate(input_ids)
        decoded_tokens = self.tokenizerb2e.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        input_ids = self.tokenizere2b(normalize(decoded_tokens), return_tensors="pt").input_ids
        generated_tokens = self.modele2b.generate(input_ids)
        decoded_tokens = self.tokenizere2b.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        if debug:
            decoded_tokens += '(bt)'
        return decoded_tokens

# Synonym Replacement

In [12]:
class SynonymReplacementAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, stopwords):
        super().__init__(clean_text)
        self.stopwords = stopwords

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        random_word_list = list(set([word for word in words if word not in self.stopwords]))
        random.shuffle(random_word_list)
        num_replaced = 0
        for random_word in random_word_list:
            synonyms = get_synonyms(random_word)
            if len(synonyms) >= 1:
                synonym = random.choice(list(synonyms))
                new_words = [synonym if word == random_word else word for word in new_words]
                num_replaced += 1
            if num_replaced >= n:
                break

        output = ' '.join(new_words)
        if debug:
            output += "(sr)"
        return output

# MLM

In [13]:
class MLMAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text, model_name="csebuetnlp/banglabert_generator"):
        super().__init__(clean_text)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMaskedLM.from_pretrained(model_name)

    def augment(self, text, num_mask=3, debug=False):
        token_ids = self.tokenizer.encode(text, return_tensors='pt')[0]
        indices_to_mask = random.sample(range(1, len(token_ids) - 1), num_mask)
        masked_token_ids = [self.tokenizer.mask_token_id if i in indices_to_mask else token_id for i, token_id in enumerate(token_ids)]
        masked_text = self.tokenizer.decode(masked_token_ids)

        input_ids = self.tokenizer(masked_text, return_tensors='pt').input_ids
        predictions = self.model(input_ids).logits
        masked_indices = (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        for idx in masked_indices:
            predicted_token_id = predictions[0, idx].topk(1).indices.item()
            input_ids[0, idx] = predicted_token_id

        unmasked_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        if debug:
            unmasked_text += "(mlm)"
        return unmasked_text

# Random Deletion

In [14]:
# Probability based
class RandomDeletionAugmentor(BaseTextAugmentor):
    def augment(self, text, p=0.3, debug=False):
        words = text.split()
        if len(words) == 1:
            return words

        new_words = []
        for word in words:
            r = random.uniform(0, 1)
            if r > p:
                new_words.append(word)

        if len(new_words) == 0:
            rand_int = random.randint(0, len(words) - 1)
            return [words[rand_int]]

        output = ' '.join(new_words)
        if debug:
            output += "(rd)"
        return output

In [15]:
## Word based
# class RandomDeletionAugmentor(BaseTextAugmentor):
#     def augment(self, text, n=1, debug=False):
#         words = text.split()
#         total_words = len(words)

#         # If there's only one word, or no words to delete, return the original text
#         if total_words == 1 or n == 0:
#             return text

#         # Ensure that the number of words to delete does not exceed the total number of words
#         n = min(int(n), total_words)  # Convert n to an integer

#         # Randomly select indices of words to delete
#         indices_to_delete = set(random.sample(range(total_words), n))

#         new_words = [word for i, word in enumerate(words) if i not in indices_to_delete]

#         # If all words are deleted, randomly select one to return
#         if len(new_words) == 0:
#             return words[random.randint(0, total_words - 1)]

#         output = ' '.join(new_words)
#         if debug:
#             output += "(rd)"
#         return output


# Random Swap

In [16]:
class RandomSwapAugmentor(BaseTextAugmentor):
    def swap_word(self, new_words):
        random_idx_1 = random.randint(0, len(new_words) - 1)
        random_idx_2 = random.randint(0, len(new_words) - 1)
        while random_idx_2 == random_idx_1:
            random_idx_2 = random.randint(0, len(new_words) - 1)
        new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
        return new_words

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        for _ in range(n):
            new_words = self.swap_word(new_words)
        output =  ' '.join(new_words)
        if debug:
            output += "(rs)"
        return output

# Random Insertion

In [17]:
class RandomInsertionAugmentor(BaseTextAugmentor):
    def __init__(self, clean_text):
        super().__init__(clean_text)

    def add_word(self, new_words):
        synonyms = []
        counter = 0
        while len(synonyms) < 1:
            random_word = new_words[random.randint(0, len(new_words)-1)]
            synonyms = get_synonyms(random_word)
            counter += 1
            if counter >= 10:
                return
        random_synonym = synonyms[0]
        random_idx = random.randint(0, len(new_words)-1)
        new_words.insert(random_idx, random_synonym)

    def augment(self, text, n, debug=False):
        words = text.split()
        new_words = words.copy()
        for _ in range(n):
            self.add_word(new_words)
        output =  ' '.join(new_words)
        if debug:
            output +=  ' '.join(new_words) + "(ri)"
        return output

# Pipeline

In [18]:
import random

# Assuming CleanText and other Augmentor classes are defined elsewhere
# alpha = percentage of words changed in a sentence
# n = alpha * length
# weights = proportion of augmented text generated by each technique

class TextAugmentationPipeline:
    def __init__(self, augmentors):
        self.augmentors = augmentors

    def augment(self, weights, text, num_aug=9, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, alpha_mlm=0.1, p_paraphrase=0.1, p_bt=0.1, debug=False):
        text = self.augmentors[0].clean_text.remove_punctuations(text)
        words = text.split()
        num_words = len(words)

        num_new_per_technique = []
        for w in weights:
          # if w==0:
          #   num_new_per_technique.append(0)
          #   continue
          num_new_per_technique.append(int(w*num_aug))

        augmented_sentences = []
        # num_new_per_technique = int(num_aug / len(self.augmentors)) + 1

        augmented_sentence_type = {
            0: "Synonym Replacement",
            1: "Random Deletion",
            2: "Random Swap",
            3: "Random Insertion",
            4: "Bangla Paraphrase",
            5: "BT Augmentation",
            6: "MLM Augmentation"
        }

        if alpha_sr > 0:
            n_sr = max(1, int(alpha_sr * num_words))
            for _ in range(num_new_per_technique[0]):
                augmented_sentences.append((self.augmentors[0].augment(text, n_sr, debug), augmented_sentence_type[0]))

        if p_rd > 0:
            n_rd = max(1, int(p_rd * num_words))
            for _ in range(num_new_per_technique[1]):
                augmented_sentences.append((self.augmentors[1].augment(text, p_rd, debug), augmented_sentence_type[1]))

        if alpha_rs > 0:
            n_rs = max(1, int(alpha_rs * num_words))
            for _ in range(num_new_per_technique[2]):
                augmented_sentences.append((self.augmentors[2].augment(text, n_rs, debug), augmented_sentence_type[2]))

        if alpha_ri > 0:
            n_ri = max(1, int(alpha_ri * num_words))
            for _ in range(num_new_per_technique[3]):
                augmented_sentences.append((self.augmentors[3].augment(text, n_ri, debug), augmented_sentence_type[3]))

        if p_paraphrase > 0:
            for _ in range(num_new_per_technique[4]):
                augmented_sentences.append((self.augmentors[4].augment(text, debug), augmented_sentence_type[4]))

        if p_bt > 0:
            for _ in range(num_new_per_technique[5]):
                augmented_sentences.append((self.augmentors[5].augment(text, debug), augmented_sentence_type[5]))

        if alpha_mlm > 0:
            n_mlm = max(2, int(alpha_mlm * num_words))
            for _ in range(num_new_per_technique[6]):
                augmented_sentences.append((self.augmentors[6].augment(text, n_mlm, debug), augmented_sentence_type[6]))

        random.shuffle(augmented_sentences)

        if num_aug >= 1:
            augmented_sentences = augmented_sentences[:num_aug]
        else:
            keep_prob = num_aug / len(augmented_sentences)
            augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

        if debug:
            augmented_sentences.append((text + "(original)", "Original"))

        return augmented_sentences

# Assuming CleanText and augmentor classes are defined/imported


In [19]:
# Usage
clean_text = CleanText()
stopwords = set(corpus.stopwords)

augmentors = [
    SynonymReplacementAugmentor(clean_text, stopwords),
    RandomDeletionAugmentor(clean_text),
    RandomSwapAugmentor(clean_text),
    RandomInsertionAugmentor(clean_text),
    BanglaParaphraseAugmentor(clean_text),
    BTAugmentor(clean_text),
    MLMAugmentor(clean_text)
]

pipeline = TextAugmentationPipeline(augmentors)


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/140M [00:00<?, ?B/s]

# Single sentence debug

In [20]:
text = "সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।"
# ct = CleanText()
# ctext = ct.remove_punctuations(text)
w = [0.1, 0, 0.1, 0.1, 0.4, 0.3, 0]
augmented_sentences = pipeline.augment(w, text, num_aug = 30, debug=False)

Extracting: bangla_word2vec_gen4.zip:   0%|          | 0/8 [00:00<?, ?it/s]

In [21]:
len(augmented_sentences)

30

In [22]:
for augmented_sentence, augmentation_type in augmented_sentences:
    print(f"{augmented_sentence}: {augmentation_type}")

সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।: Bangla Paraphrase
সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।: BT Augmentation
সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।: BT Augmentation
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে আনার এই কাজ টা করেছে: Random Insertion
টা তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ সঠিক করেছে: Random Swap
সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।: Bangla Paraphrase
সঠিক তদন্ত বিচারের হবে করতে আওতায় আনতে হবে যে এই কাজ টা করেছে: Random Swap
সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।: Bangla Paraphrase
সঠিক তদন্ত অবশ্যই করতে হবে এবং যে ব্যক্তি এটা করেছে তাকে বিচারের আওতায় আনতে হবে।: BT Augmentation
সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে।: Bangla Paraphrase
সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ গ্রেপ্তারের টা করেছে: Random Insertion
যথাযথ তদন্ত 

# SacreBLEU

In [23]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.4 MB/s eta 0:00:00


In [24]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1b54906494f646935c37ddd7aa31ebdac8d68cf9fe2497757ed00e9b88deb19a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [25]:
import sacrebleu
from sentence_transformers import SentenceTransformer, util
import torch

# Function to calculate cosine similarity
def cosine_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0)

# Initialize the SBERT model
model_name = 'l3cube-pune/bengali-sentence-bert-nli'
model = SentenceTransformer(model_name)

# Example reference sentence in Bangla
reference = text  # This is a hypothetical example sentence in Bangla

# List of augmented candidate sentences in Bangla along with Augtype
candidates = augmented_sentences

# Compute BLEU score for each candidate
sacrebleu_scores = [sacrebleu.sentence_bleu(candidate[0], [reference]).score for candidate in candidates]

# Encode the reference sentence to get its embedding
reference_embedding = model.encode(reference, convert_to_tensor=True)

# Encode candidate sentences to get their embeddings
candidate_sentences = [candidate[0] for candidate in candidates]
candidate_embeddings = model.encode(candidate_sentences, convert_to_tensor=True)

# Calculate Cosine Similarity
cosine_similarities = [cosine_similarity(reference_embedding, embedding).item() for embedding in candidate_embeddings]

# Pair and sort the candidate sentences based on Cosine Similarity scores
sentence_similarity_pairs = list(zip(candidates, sacrebleu_scores, cosine_similarities))
sorted_pairs = sorted(sentence_similarity_pairs, key=lambda x: x[2], reverse=True)

# Print the sorted candidate sentences, their Augtype, BLEU, and Cosine Similarity scores
for i, (candidate, score, cos_sim) in enumerate(sorted_pairs):
    print(f"Candidate {i + 1}: {candidate[0]} - Augtype: {candidate[1]}, SacreBLEU Score: {score}, SBERT Cosine Similarity: {cos_sim}")


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Candidate 1: করে সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে - Augtype: Random Insertion, SacreBLEU Score: 61.62607099729587, SBERT Cosine Similarity: 0.988017201423645
Candidate 2: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে। - Augtype: Bangla Paraphrase, SacreBLEU Score: 37.59663529467017, SBERT Cosine Similarity: 0.9824073910713196
Candidate 3: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে। - Augtype: Bangla Paraphrase, SacreBLEU Score: 37.59663529467017, SBERT Cosine Similarity: 0.9824073910713196
Candidate 4: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে। - Augtype: Bangla Paraphrase, SacreBLEU Score: 37.59663529467017, SBERT Cosine Similarity: 0.9824073910713196
Candidate 5: সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায় আনতে হবে যে এটি করেছে। - Augtype: Bangla Paraphrase, SacreBLEU Score: 37.59663529467017, SBERT Cosine Similarity: 0.9824073910713196
Candidate 6: সঠিক তদন্ত করতে হবে এবং

# Dataframe output

In [28]:
    reference = text  # Reference sentence in Bangla

    candidates = augmented_sentences

    sacrebleu_scores = [sacrebleu.sentence_bleu(candidate[0], [reference]).score for candidate in candidates]

    reference_embedding = model.encode(reference, convert_to_tensor=True)

    candidate_sentences = [candidate[0] for candidate in candidates]
    candidate_embeddings = model.encode(candidate_sentences, convert_to_tensor=True)

    cosine_similarities = [cosine_similarity(reference_embedding, embedding).item() for embedding in candidate_embeddings]

    sentence_similarity_pairs = list(zip(candidates, sacrebleu_scores, cosine_similarities))
    sorted_pairs = sorted(sentence_similarity_pairs, key=lambda x: x[2], reverse=True)

    for i, (candidate, score, cos_sim) in enumerate(sorted_pairs):
        results.append({
            "augmented_text": candidate[0],
            "label": row["label"],
            "Augtype": candidate[1],
            "SacreBLEU": score,
            "SBERT Cosine Similarity": cos_sim,
            "Original ID": row["ID"]
        })

output_df = pd.DataFrame(results)

# Display the DataFrame or perform further operations as required
print(output_df)


                                       augmented_text    label  \
0   করে সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হব...  neutral   
1   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
2   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
3   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
4   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
5   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
6   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
7   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
8   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
9   সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
10  সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
11  সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
12  সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...  neutral   
13  সঠিক তদন্ত করতে হবে বিচারের আওতা আনতে হবে যে এ...  neutral   
14  যথাযথ 

In [29]:
# Print the output DataFrame
output_df

,augmented_text,label,Augtype,SacreBLEU,SBERT Cosine Similarity,Original ID
0,করে সঠিক তদন্ত করতে হবে বিচারের আওতায় আনতে হব...,neutral,Random Insertion,61.626071,0.988017,1
1,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
2,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
3,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
4,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
5,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
6,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
7,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
8,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
9,সঠিক তদন্ত করতে হবে এবং এই কাজটি বিচারের আওতায...,neutral,Bangla Paraphrase,37.596635,0.982407,1
